In [1]:
import os
import numpy as np

In [2]:
all_configs = []
all_configs.append({
    "diversity_loss_coef": 1.0,
    "entropy_loss_coef": 1.0,
})
all_configs.append({
    "diversity_loss_coef": 5.0,
    "entropy_loss_coef": 5.0,
})
all_configs.append({
    "diversity_loss_coef": 10.0,
    "entropy_loss_coef": 10.0,
})

In [3]:
jax_data_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/"
scripts_dir = "/global/home/users/aniketh/promoter/"

oracle_test_data_path = os.path.join(jax_data_dir, "finetune_data.pkl")
assert os.path.exists(oracle_test_data_path)

saved_models_dir = os.path.join(jax_data_dir, "saved_DEN_models_v7_TITAN")
if not os.path.exists(saved_models_dir):
    os.mkdir(saved_models_dir)
assert os.path.exists(saved_models_dir)

wandb_dir = os.path.join(jax_data_dir, "wandb_v7_TITAN")
if not os.path.exists(wandb_dir):
    os.mkdir(wandb_dir)
assert os.path.exists(wandb_dir)

bash_scripts_dir = "/global/home/users/aniketh/promoter/promoter/run_DENs_v7_TITAN_bash_scripts"
if not os.path.exists(bash_scripts_dir):
    os.mkdir(bash_scripts_dir)
    
total_count = len(os.listdir(os.path.join(jax_data_dir, "trained_predictors_new_loss"))) * len(["THP1", "Jurkat", "K562"]) * len(all_configs)

num_splits = 2
runs_per_job = int(np.ceil(total_count / num_splits))
job_cnt = 0
cur_cnt = 0

target_cell_exp_weight = {"THP1": 2, "Jurkat": 2, "K562": 2}

slurm_prefix = open("slurm_prefix.txt", "r").readlines()

g = open(os.path.join(bash_scripts_dir, f"train_all_DENs_j{job_cnt}.sh"), "w+")
for line in slurm_prefix:
    g.write(line)
    
rem_count = 0

for model in sorted(os.listdir(os.path.join(jax_data_dir, "trained_predictors_new_loss"))):
    for diff_exp_cell_ind, cell in enumerate(["THP1", "Jurkat", "K562"]):
        for config in all_configs:
            diversity_loss_coef = config["diversity_loss_coef"]
            entropy_loss_coef = config["entropy_loss_coef"]
            
            experiment_id = f"{cell}_using_{model}_div_coef_{diversity_loss_coef}_ent_coef_{entropy_loss_coef}"
            pretrained_predictor_path = os.path.join(jax_data_dir, "trained_predictors_new_loss", model)
            
            if os.path.exists(os.path.join(saved_models_dir, experiment_id, "final_sequences.npy")):
                continue
                
            rem_count += 1
        
            if cur_cnt == runs_per_job:
                job_cnt += 1
                cur_cnt = 0
                g.close()
                g = open(os.path.join(bash_scripts_dir, f"train_all_DENs_j{job_cnt}.sh"), "w+")
                for line in slurm_prefix:
                    g.write(line)
                print("Created new job split")

            cmd = f"sbatch --requeue --export=scripts_dir={scripts_dir},pretrained_predictor_path={pretrained_predictor_path},diff_exp_cell_ind={diff_exp_cell_ind},diversity_loss_coef={diversity_loss_coef},entropy_loss_coef={entropy_loss_coef},oracle_test_data_path={oracle_test_data_path},saved_models_dir={saved_models_dir},wandb_dir={wandb_dir},experiment_id={experiment_id} run_DENs_v7_TITAN.sh"
            os.system(cmd)
            print(cmd)
            print()

            f = open(os.path.join(bash_scripts_dir, experiment_id + ".sh"), "w+")
            f.write("#!/bin/bash\n")
            all_exports = f"scripts_dir={scripts_dir},pretrained_predictor_path={pretrained_predictor_path},diff_exp_cell_ind={diff_exp_cell_ind},diversity_loss_coef={diversity_loss_coef},entropy_loss_coef={entropy_loss_coef},oracle_test_data_path={oracle_test_data_path},saved_models_dir={saved_models_dir},wandb_dir={wandb_dir},experiment_id={experiment_id}"
            for ex in all_exports.split(","):
                var, val = ex.split("=")
                print(f"export {var}={val}")
                f.write(f"export {var}={val}\n")
            print("bash {}".format(os.path.join(scripts_dir, "promoter/run_DENs_v7_TITAN.sh")))
            f.write("bash {}\n".format(os.path.join(scripts_dir, "promoter/run_DENs_v7_TITAN.sh")))
            f.close()

            g.write("bash {}".format(os.path.join(bash_scripts_dir, experiment_id + ".sh\n")))        
            print()

            cur_cnt += 1
g.close()

Submitted batch job 14511613
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.03_total_rounds_2.pkl,diff_exp_cell_ind=0,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=THP1_using_coms_0.03_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.03_total_rounds_2.pkl
export diff_exp_cell_ind=0
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_

Submitted batch job 14511619
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.03_total_rounds_2.pkl,diff_exp_cell_ind=2,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=K562_using_coms_0.03_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.03_total_rounds_2.pkl
export diff_exp_cell_ind=2
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_

Submitted batch job 14511625
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.03_total_rounds_5.pkl,diff_exp_cell_ind=1,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=Jurkat_using_coms_0.03_total_rounds_5.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.03_total_rounds_5.pkl
export diff_exp_cell_ind=1
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracl

Submitted batch job 14511631
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.0_total_rounds_2.pkl,diff_exp_cell_ind=0,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=THP1_using_coms_0.0_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.0_total_rounds_2.pkl
export diff_exp_cell_ind=0
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_tes

Submitted batch job 14511637
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.0_total_rounds_2.pkl,diff_exp_cell_ind=2,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=K562_using_coms_0.0_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.0_total_rounds_2.pkl
export diff_exp_cell_ind=2
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_tes

...

Submitted batch job 14511655
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.1_total_rounds_2.pkl,diff_exp_cell_ind=2,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=K562_using_coms_0.1_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.1_total_rounds_2.pkl
export diff_exp_cell_ind=2
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_tes

Submitted batch job 14511661
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.1_total_rounds_5.pkl,diff_exp_cell_ind=1,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=Jurkat_using_coms_0.1_total_rounds_5.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.1_total_rounds_5.pkl
export diff_exp_cell_ind=1
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_t

Submitted batch job 14511667
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.3_total_rounds_2.pkl,diff_exp_cell_ind=0,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=THP1_using_coms_0.3_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.3_total_rounds_2.pkl
export diff_exp_cell_ind=0
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_tes

Submitted batch job 14511673
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.3_total_rounds_2.pkl,diff_exp_cell_ind=2,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=K562_using_coms_0.3_total_rounds_2.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.3_total_rounds_2.pkl
export diff_exp_cell_ind=2
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_tes

Submitted batch job 14511679
sbatch --requeue --export=scripts_dir=/global/home/users/aniketh/promoter/,pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.3_total_rounds_5.pkl,diff_exp_cell_ind=1,diversity_loss_coef=1.0,entropy_loss_coef=1.0,oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl,saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v7_TITAN,wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v7_TITAN,experiment_id=Jurkat_using_coms_0.3_total_rounds_5.pkl_div_coef_1.0_ent_coef_1.0 run_DENs_v7_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_new_loss/coms_0.3_total_rounds_5.pkl
export diff_exp_cell_ind=1
export diversity_loss_coef=1.0
export entropy_loss_coef=1.0
export oracle_t

In [4]:
total_count

72

In [5]:
rem_count

72